---
title: Downloading Climate Data from ERA5-Land
short_title: ERA5 Climate Data
---

In this notebook we will demonstrate how we can use Climate Tools and `dhis2eo` to retrieve climate data for a set of DHIS2 organisation units, based on the [ERA5-Land dataset](https://cds.climate.copernicus.eu/datasets/reanalysis-era5-land), hosted at the [ECMWF Climate Data Store (CDS)](https://cds.climate.copernicus.eu/datasets). 

**Important:** Make sure you have followed [these instructions](api-authentication.md) to authenticate and allow API access the CDS portal. 

--------------------------------------------
## What you need

We start by importing the necessary libraries:

In [1]:
import geopandas as gpd
import dhis2eo
from dhis2eo.data.cds import era5_land
from dhis2eo import utils

In order to know which geographical area we should download data for, we also load our Sierra Leone organisation unit GeoJSON file downloaded from DHIS2. Alternative, see our [guide for how to fetch organisation units directly from your DHIS2 instance](../../org-units/download-web-api.ipynb). 

In [2]:
org_units = gpd.read_file('../../data/sierra-leone-districts.geojson')
org_units

,type,id,name,hasCoordinatesDown,hasCoordinatesUp,level,grandParentParentGraph,grandParentId,parentGraph,parentId,parentName,dimensions,weight,geometry
0,Polygon,O6uvpzGd5pu,Bo,True,False,2,,,ImspTQPwCqd,ImspTQPwCqd,Sierra Leone,{ },1,"POLYGON ((-11.5914 8.4875, -11.5906 8.4769, -1..."
1,Polygon,fdc6uOvgoji,Bombali,True,False,2,,,ImspTQPwCqd,ImspTQPwCqd,Sierra Leone,{ },1,"POLYGON ((-11.8091 9.2032, -11.8102 9.1944, -1..."
2,MultiPolygon,lc3eMKXaEfw,Bonthe,True,False,2,,,ImspTQPwCqd,ImspTQPwCqd,Sierra Leone,{ },1,"MULTIPOLYGON (((-12.5568 7.3832, -12.5574 7.38..."
3,Polygon,jUb8gELQApl,Kailahun,True,False,2,,,ImspTQPwCqd,ImspTQPwCqd,Sierra Leone,{ },1,"POLYGON ((-10.7972 7.5866, -10.8002 7.5878, -1..."
4,MultiPolygon,PMa2VCrupOd,Kambia,True,False,2,,,ImspTQPwCqd,ImspTQPwCqd,Sierra Leone,{ },1,"MULTIPOLYGON (((-13.1349 8.8471, -13.1343 8.84..."
5,Polygon,kJq2mPyFEHo,Kenema,True,False,2,,,ImspTQPwCqd,ImspTQPwCqd,Sierra Leone,{ },1,"POLYGON ((-11.3596 8.5317, -11.3513 8.5234, -1..."
6,Polygon,qhqAxPSTUXp,Koinadugu,True,False,2,,,ImspTQPwCqd,ImspTQPwCqd,Sierra Leone,{ },1,"POLYGON ((-10.585 9.0434, -10.5877 9.0432, -10..."
7,Polygon,Vth0fbpFcsO,Kono,True,False,2,,,ImspTQPwCqd,ImspTQPwCqd,Sierra Leone,{ },1,"POLYGON ((-10.585 9.0434, -10.5848 9.0432, -10..."
8,MultiPolygon,jmIPBj66vD6,Moyamba,True,False,2,,,ImspTQPwCqd,ImspTQPwCqd,Sierra Leone,{ },1,"MULTIPOLYGON (((-12.6351 7.6613, -12.6346 7.66..."
9,MultiPolygon,TEQlaapDQoK,Port Loko,True,False,2,,,ImspTQPwCqd,ImspTQPwCqd,Sierra Leone,{ },1,"MULTIPOLYGON (((-13.119 8.4718, -13.1174 8.470..."


--------

## Downloading ERA5-Land data

The `dhis2eo.data.cds.era5_land` module allows you to easily download climate data for different time periods from the ERA5-Land dataset. 

### Hourly data

To import daily ERA5-Land data into DHIS2, we recommend downloading [ERA5-Land data at hourly levels](https://cds.climate.copernicus.eu/datasets/reanalysis-era5-land), and then [aggregating up to the daily level](../../aggregation/time-period-aggregation.ipynb). This is because the [ERA5-Land at daily levels](https://cds.climate.copernicus.eu/datasets/derived-era5-land-daily-statistics) does not include the precipitation variable. 

Downloading this data can be done by calling the function `dhis2eo.data.cds.era5_land.hourly.get(...)`. By default, this downloads the most commonly requested ERA5 climate variables: 2m temperature and total precipitation, but you can also set the `variables` parameters to any of the ERA5-Land variables. 

Due to the large amounts of data, this dataset only allows downloading one month at a time. We therefore need to loop over each year and month, and call the download function for each `year` and `month` value. 

For the purpose of this notebook, let's download data for the last 3 months: 

In [3]:
start_year = 2025
start_month = 7
end_year = 2025
end_month = 9

for year,month in utils.time.iter_months(start_year, start_month, end_year, end_month):
    
    # download the climate data
    print(f'Month: {year}-{month}')
    hourly_data = era5_land.hourly.get(year=year, month=month, bbox=org_units.total_bounds)

    # do something with the data
    # e.g. save to disk, aggregate, or import to DHIS2
    # ...

Month: 2025-7
dhis2eo.data.cds.era5_land.hourly - INFO - Downloading data from CDS API...
dhis2eo.data.cds.era5_land.hourly - INFO - Request parameters: 
{"variable": ["2m_temperature", "total_precipitation"], "year": "2025", "month": ["07"], "day": ["01", "02", "03", "04", "05", "06", "07", "08", "09", "10", "11", "12", "13", "14", "15", "16", "17", "18", "19", "20", "21", "22", "23", "24", "25", "26", "27", "28", "29", "30", "31"], "time": ["00:00", "01:00", "02:00", "03:00", "04:00", "05:00", "06:00", "07:00", "08:00", "09:00", "10:00", "11:00", "12:00", "13:00", "14:00", "15:00", "16:00", "17:00", "18:00", "19:00", "20:00", "21:00", "22:00", "23:00"], "area": [10.0004, -13.3035, 6.9176, -10.2658], "data_format": "netcdf", "download_format": "unarchived"}


2025-12-02 15:26:15,126 INFO Request ID is 5138f84b-a41e-4e3b-97f2-ac946e0d5c0e
2025-12-02 15:26:15,222 INFO status has been updated to accepted
2025-12-02 15:26:20,208 INFO status has been updated to running
2025-12-02 15:30:33,937 INFO status has been updated to successful


32430075ca6e374c7f912770c72eedb1.nc:   0%|          | 0.00/3.26M [00:00<?, ?B/s]

Month: 2025-8
dhis2eo.data.cds.era5_land.hourly - INFO - Downloading data from CDS API...
dhis2eo.data.cds.era5_land.hourly - INFO - Request parameters: 
{"variable": ["2m_temperature", "total_precipitation"], "year": "2025", "month": ["08"], "day": ["01", "02", "03", "04", "05", "06", "07", "08", "09", "10", "11", "12", "13", "14", "15", "16", "17", "18", "19", "20", "21", "22", "23", "24", "25", "26", "27", "28", "29", "30", "31"], "time": ["00:00", "01:00", "02:00", "03:00", "04:00", "05:00", "06:00", "07:00", "08:00", "09:00", "10:00", "11:00", "12:00", "13:00", "14:00", "15:00", "16:00", "17:00", "18:00", "19:00", "20:00", "21:00", "22:00", "23:00"], "area": [10.0004, -13.3035, 6.9176, -10.2658], "data_format": "netcdf", "download_format": "unarchived"}


2025-12-02 15:30:40,010 INFO Request ID is c57b7cb6-ad45-4ab3-92b8-11119acb3da1
2025-12-02 15:30:40,073 INFO status has been updated to accepted
2025-12-02 15:30:48,692 INFO status has been updated to running
2025-12-02 15:34:58,896 INFO status has been updated to successful


84ee6f09e816dee25e25804b77feb8ca.nc:   0%|          | 0.00/3.26M [00:00<?, ?B/s]

Month: 2025-9
dhis2eo.data.cds.era5_land.hourly - INFO - Downloading data from CDS API...
dhis2eo.data.cds.era5_land.hourly - INFO - Request parameters: 
{"variable": ["2m_temperature", "total_precipitation"], "year": "2025", "month": ["09"], "day": ["01", "02", "03", "04", "05", "06", "07", "08", "09", "10", "11", "12", "13", "14", "15", "16", "17", "18", "19", "20", "21", "22", "23", "24", "25", "26", "27", "28", "29", "30"], "time": ["00:00", "01:00", "02:00", "03:00", "04:00", "05:00", "06:00", "07:00", "08:00", "09:00", "10:00", "11:00", "12:00", "13:00", "14:00", "15:00", "16:00", "17:00", "18:00", "19:00", "20:00", "21:00", "22:00", "23:00"], "area": [10.0004, -13.3035, 6.9176, -10.2658], "data_format": "netcdf", "download_format": "unarchived"}


2025-12-02 15:35:01,464 INFO Request ID is ffc2c8c6-45e4-4ccd-9034-828f6c459be1
2025-12-02 15:35:01,527 INFO status has been updated to accepted
2025-12-02 15:35:15,100 INFO status has been updated to running
2025-12-02 15:37:53,934 INFO status has been updated to successful


125d12f597c0ca71601915e4619794b5.nc:   0%|          | 0.00/3.14M [00:00<?, ?B/s]

To inspect the contents of the downloaded data, let's view the data of the last element in the loop: 

In [5]:
hourly_data

<xarray.Dataset> Size: 6MB
Dimensions:     (valid_time: 720, latitude: 31, longitude: 31)
Coordinates:
  * valid_time  (valid_time) datetime64[ns] 6kB 2025-09-01 ... 2025-09-30T23:...
  * latitude    (latitude) float64 248B 9.918 9.818 9.718 ... 7.118 7.018 6.917
  * longitude   (longitude) float64 248B -13.3 -13.2 -13.1 ... -10.5 -10.4 -10.3
    number      int64 8B ...
    expver      (valid_time) <U4 12kB ...
Data variables:
    t2m         (valid_time, latitude, longitude) float32 3MB ...
    tp          (valid_time, latitude, longitude) float32 3MB ...
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-11-13T19:24 GRIB to CDM+CF via cfgrib-0.9.1...

We see that this is hourly data for the month of September 2025, and contains data variables `t2m` (2m temperature), and `tp` (total precipitation). 

Note: Since DHIS2 does not deal directly with hourly data, you will also need to [aggregate from hourly to daily data](../../aggregation/time-period-aggregation.ipynb) before you can import the data to DHIS2. 

### Monthly data

Sometimes you may only be interested in monthly climate data at longer time scales. In these cases, we recommend downloading the [monthly ERA5-Land data](https://cds.climate.copernicus.eu/datasets/reanalysis-era5-land-monthly-means). 

These data can be downloaded by calling the function `dhis2eo.data.cds.era_land.monthly.get(...)`. Similarly to the hourly data, the default variables are temperature and precipitation, and can be customized be specifying the `variables` parameter. 

Since monthly data takes a lot less space, you can download all the data you need by specifying a list of `years` and a list of `months`. 

Let's download monthly data for the last 10 years:

In [4]:
years = list(range(2015, 2025+1))
months = list(range(1, 12+1))
monthly_data = era5_land.monthly.get(years=years, months=months, bbox=org_units.total_bounds)

dhis2eo.data.cds.era5_land.monthly - INFO - Downloading data from CDS API...
dhis2eo.data.cds.era5_land.monthly - INFO - Request parameters: 
{"product_type": ["monthly_averaged_reanalysis"], "variable": ["2m_temperature", "total_precipitation"], "year": ["2015", "2016", "2017", "2018", "2019", "2020", "2021", "2022", "2023", "2024", "2025"], "month": ["01", "02", "03", "04", "05", "06", "07", "08", "09", "10", "11", "12"], "time": ["00:00"], "area": [10.0004, -13.3035, 6.9176, -10.2658], "data_format": "netcdf", "download_format": "unarchived"}


2025-12-02 15:37:55,973 INFO Request ID is f7ae0c9c-9788-46e1-820f-30062d243c98
2025-12-02 15:37:56,057 INFO status has been updated to accepted
2025-12-02 15:38:17,403 INFO status has been updated to running
2025-12-02 15:38:28,982 INFO status has been updated to accepted
2025-12-02 15:38:46,151 INFO status has been updated to successful


72abe67607bae196efac518e447b11e7.nc:   0%|          | 0.00/619k [00:00<?, ?B/s]

And inspect the results:

In [8]:
monthly_data

<xarray.Dataset> Size: 1MB
Dimensions:     (valid_time: 130, latitude: 31, longitude: 31)
Coordinates:
  * valid_time  (valid_time) datetime64[ns] 1kB 2015-01-01 ... 2025-10-01
  * latitude    (latitude) float64 248B 9.918 9.818 9.718 ... 7.118 7.018 6.917
  * longitude   (longitude) float64 248B -13.3 -13.2 -13.1 ... -10.5 -10.4 -10.3
    number      int64 8B ...
    expver      (valid_time) <U4 2kB ...
Data variables:
    t2m         (valid_time, latitude, longitude) float32 500kB ...
    tp          (valid_time, latitude, longitude) float32 500kB ...
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-11-13T19:00 GRIB to CDM+CF via cfgrib-0.9.1...

We see the result contains 130 months (over 10 years) of data for temperature and precipitation variables. 

#### Important note about monthly precipitation!

The [monthly version of ERA5-Land](https://cds.climate.copernicus.eu/datasets/reanalysis-era5-land-monthly-means) reports all variables as daily averages. For some of the variables such as precipitation, we are instead interested in the total precipitation for the entire month. This can be done by multiplying the average daily precipitation (`tp`) with the number of days per month (xarray date fields provide this as an attribute `.dt.days_in_month`): 

In [ ]:
total_precip = monthly_data['tp'] * monthly_data.valid_time.dt.days_in_month

## Next steps

This notebook has showed how to download both hourly and monthly ERA5-Land climate data. For guidance on what to do with the data after downloading, see: 

* [Aggregate the data](../../aggregation/intro.md).
* [Import the aggregated data to DHIS2](../../import-data/intro.md).

If you want to skip to the end, you can also check out [this example workflow](../../import-data/example-import-dhis2-daily.ipynb) that includes all the steps needed to download and import ERA5-Land data from start to finish. 